# Other Code

Given a fits image of a survey, calculate the area of the survey in degrees^2. The fits image is in pixels and the CDELT1 value on the header is used to rescale to physical units.

In [ ]:
def calculate_survey_area(fits_file):
    # Open the FITS file and extract the header
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        data = hdul[0].data
        
        # Ensure data is at least 2D (handle multi-dimensional FITS files)
        if data.ndim > 2:
            data = data[0]  # Take the first frame if it's a multi-frame FITS
        
        # Get image dimensions
        height, width = data.shape[-2:]  # Ensure correct shape indexing
        total_pixels = width * height
        
        # Extract pixel scale from header (if available)
        if 'PIXSCALE' in header:
            pixel_scale = header['PIXSCALE']  # in arcsec/pixel
        elif 'CDELT1' in header and 'CDELT2' in header:
            if header['CUNIT1'] == 'deg' and header['CUNIT2'] == 'deg': # Check if units are in degrees
                pixel_scale = np.abs(header['CDELT1']) * 3600  # Convert from decimal degrees to arcseconds
        else:
            raise ValueError("Pixel scale not found in FITS header. Check metadata or specify manually.")
        
        # Compute the survey area in square arcseconds
        area_arcsec2 = total_pixels * (pixel_scale ** 2)
        
        # Convert to square degrees
        area_deg2 = area_arcsec2 / (3600 ** 2)
        
    print(f"Total pixels: {total_pixels}")
    print(f"Pixel scale (arcsec/pixel): {pixel_scale}")
    print(f"Survey area (deg^2): {area_deg2}")

    return area_deg2

In [ ]:
area_emu_survey = calculate_survey_area(emu_mosaic_path)

In [ ]:
# Create a list of the field name and the corresponding SkyCoord object of the center of the field
emu_tiles_dic = [[emu_html_df['field_name'][i], emu_center_coords[i]] for i in range(len(emu_html_df))]

# Create a list of valid field names and corresponding SkyCoord object
emu_tiles_valid_dic = [[emu_html_df['field_name'][i], emu_center_coords[i]] for i in range(len(emu_html_df)) if has_sbid[i]]

In [ ]:
# polygons for area analysis
emu_tiles_deg = []
emu_tiles_rad = []

# polygons for plotting in degrees and radians
emu_tiles_plot_deg = []
emu_tiles_plot_rad = []

# polygons with valid tiles for plotting in radians
emu_tiles_plot_rad_valid = []

for i in range(len(emu_tiles_df)):
    ra_deg = emu_tiles_df['ra_deg'][i]
    dec_deg = emu_tiles_df['dec_deg'][i]
    
    # Size of the tile in degrees
    size = 5 
    half_size = size / 2

    # Define the corners of the tile in degrees
    corners_deg = [
        (ra_deg - half_size, dec_deg - half_size),
        (ra_deg + half_size, dec_deg - half_size),
        (ra_deg + half_size, dec_deg + half_size),
        (ra_deg - half_size, dec_deg + half_size)
    ]

    # Shift RA values to center the plot at RA = 0 and convert to radians
    corners_rad = [(  np.radians(ra - 360 if ra > 180 else ra ), np.radians(dec)) for ra, dec in corners_deg]

    # Define the color of the tiles, if the SBID is missing, color it red, if not, color it blue
    color = 'red' if pd.isna(emu_tiles_df['sbid'][i]) else 'blue'

    # Append to lists
    emu_tiles_deg.append(splyPolygon(corners_deg))
    emu_tiles_rad.append(splyPolygon(corners_rad))
    if color == 'blue':
        emu_tiles_plot_rad_valid.append(mplPolygon(corners_rad))
    emu_tiles_plot_deg.append(mplPolygon(corners_deg, closed=True, edgecolor=color, facecolor='none'))
    emu_tiles_plot_rad.append(mplPolygon(corners_rad, closed=True, edgecolor=color, facecolor='none'))

In [ ]:
# create a list of DES DR1 tiles

des_tiles_deg = []
des_tiles_rad = []
des_tiles_plot_deg = []
des_tiles_plot_rad = []

for i in range(len(des_tiles_df)):

    corners_deg = [
        (des_tiles_df['rac1'][i],des_tiles_df['decc1'][i]),
        (des_tiles_df['rac2'][i],des_tiles_df['decc2'][i]),
        (des_tiles_df['rac3'][i],des_tiles_df['decc3'][i]),
        (des_tiles_df['rac4'][i],des_tiles_df['decc4'][i])
    ]

    # Shift RA values to center the plot at RA = 0 and convert to radians
    corners_rad = [(  np.radians(ra - 360 if ra > 180 else ra ), np.radians(dec)) for ra, dec in corners_deg]

    # Append to lists
    des_tiles_deg.append(splyPolygon(corners_deg))
    des_tiles_rad.append(splyPolygon(corners_rad))
    des_tiles_plot_deg.append(mplPolygon(corners_deg, closed=True, edgecolor='purple', facecolor='none'))
    des_tiles_plot_rad.append(mplPolygon(corners_rad, closed=True, edgecolor='purple', facecolor='none'))

    print(des_tiles_deg)